#**ARIMA MODEL**

#**Preprocessing**

In [10]:
#!pip install pmdarima
#Should be run once so that ARIMA can run on Colab
import pandas.testing as tm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
!pip install pmdarima
from pmdarima.utils import diff_inv
from pmdarima.arima.utils import ndiffs
from pmdarima import auto_arima
from math import sqrt
df=pd.read_csv('Data/building-electrical-energy-daily-use-ubcv-2019-2021.csv');
temperature=pd.read_csv('Data/Temperature.csv')
columns = df.columns;
df.head()


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
for i in range(1,len(df. columns)-1):
    df[columns[i]] = df[columns[i]].fillna(df[columns[i]].mean());

In [ ]:
df["Total Energy Consumption"] = df[columns[1:]].sum(axis = 1);

In [ ]:
for i in range(1,len(columns)):
    df = df.drop(columns[i], axis = 1)

In [ ]:
for i in range(0,len(df["Date"])):
    df.loc[ i,'Date']= datetime.strptime(df["Date"].loc[i], '%m/%d/%Y');

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])

#Date features
df["Day"] = df["Date"].dt.dayofweek
df["Month"] = df["Date"].dt.month
df["Year"] = df["Date"].dt.year
df["Q"] = df["Date"].dt.quarter
df["Dayofyear"] = df["Date"].dt.dayofyear
df["Dayofmonth"] = df["Date"].dt.day
df["Weekofyear"] = df["Date"].dt.isocalendar().week
#df["Temperature"]=temperature["T2MDEW"].astype('int')

In [ ]:
df["Holiday"] = 0;
for i in range(0,len(df["Dayofmonth"])):
    if(df['Dayofmonth'][i] == 1 and df["Month"][i] == 1):
        df.loc[ i, 'Holiday']  = 1
        
    elif(df['Dayofmonth'][i] == 21 and df["Month"][i] == 2):
        df.loc[ i, 'Holiday']  = 1
        
    elif(df['Dayofmonth'][i] == 15 and df["Month"][i] == 4):
        df.loc[ i, 'Holiday']  = 1
        
    elif(df['Dayofmonth'][i] == 18 and df["Month"][i] == 4):
        df.loc[ i, 'Holiday']  = 1
        
    elif(df['Dayofmonth'][i] == 23 and df["Month"][i] == 5):
        df.loc[ i, 'Holiday']  = 1
        
    elif(df['Dayofmonth'][i] == 1 and df["Month"][i] == 7):
        df.loc[ i, 'Holiday']  = 1
        
    elif(df['Dayofmonth'][i] == 1 and df["Month"][i] == 8):
        df.loc[ i, 'Holiday']  = 1
        
    elif(df['Dayofmonth'][i] == 5 and df["Month"][i] == 9):
        df.loc[ i, 'Holiday']  = 1
        
    elif(df['Dayofmonth'][i] == 10 and df["Month"][i] == 10):
        df.loc[ i, 'Holiday']  = 1
        
    elif(df['Dayofmonth'][i] == 25 and df["Month"][i] == 12):
        df.loc[ i, 'Holiday']  = 1
        
    elif(df['Dayofmonth'][i] == 26 and df["Month"][i] == 12):
        df.loc[ i, 'Holiday']  = 1
        
        
df.index = df["Date"]
df = df.drop(["Date"],axis=1)

In [ ]:
columns=["Day","Month","Q","Dayofmonth","Weekofyear"]
df[columns] = (df[columns]-df[columns].mean())/(df[columns].std())
df["Total Energy Consumption"] = df["Total Energy Consumption"] / 1000000

In [ ]:
df.head()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
cols=["Day","Year","Dayofyear","Month","Q","Dayofmonth","Weekofyear","Holiday"]
for i in range(0,len(cols)):
    df = df.drop(cols[i], axis = 1)

#**Define Train and Test**

In [ ]:
train = df.iloc[:550] #Everything before 2021 
test = df.iloc[550:] #Everything after 2021
#print(train.shape,test.shape)
print(test)

#**FINDING CORRELATION BETWEEN ORIGINAL AND DIFFERENCES**

In [ ]:
#This is used to find the d value

col=train
plt.rcParams.update({'figure.figsize':(25,10), 'figure.dpi':150})
fig, axes = plt.subplots(2, 2, sharex=False)

#plot original series with its ACF
axes[0, 0].plot(col);axes[0,0].set_title('Original Series')
plot_acf(col, ax=axes[0,1],lags=150)

#plot 1st difference with its ACF
axes[1,0].plot(col.diff()); axes[1,0].set_title('First Difference')
plot_acf(col.diff().dropna(), ax=axes[1,1],lags=150)

plt.show()
#We can see all the lags by replacing 150 with col.shape[0]-1 for original series and col.shape[0]-2 for 1st differencing

In [ ]:
#Find p-value to check if we need more differencing

#For original series
result=adfuller(col.dropna())
print("p-value: ",result[1])

#For first difference
result=adfuller(col.diff().dropna())
print("p-value: ", result[1])

#Perform ADF test to make sure how much differencing we need
ndiffs(col, test='adf') #We get d=1 here which proves our point

#So, take d=1 since 9.35e-08 < 0.05 and fastest converging to 0 as a first assumption

In [ ]:
#This is used to find the p value

plt.rcParams.update({'figure.figsize':(25,5), 'figure.dpi':120})
fig, axes = plt.subplots(1, 2, sharex=False)

#Here, we plot the PACF of the first difference since we already decided to use d=1
axes[0].plot(col.diff()); axes[0].set_title('First Difference')
plot_pacf(col.diff().dropna(), ax=axes[1], lags=150)


plt.show()

#So, take p = 5 since we notice at least 5 significant jumps over the confidence level in the PACF. 

We can see a number of lags in the ACF crossing the threshold by well above the line, so choose q=4 for now since there seems to be 4 significant levels above the confidence threshold.  

So, we have ARIMA(4,1,5)

#**Implement Intuition ARIMA model**

In [ ]:
'''intuition_arima_model = ARIMA(col,order=(4,1,5))
intuition_arima_model_fit = intuition_arima_model.fit()
warnings.filterwarnings("ignore")
print(intuition_arima_model_fit.summary())'''

#**Residuals and Density**

In [ ]:
'''resids = pd.DataFrame(intuition_arima_model_fit.resid)
fig, ax = plt.subplots(1,2)

resids.plot(title="Residuals", ax=ax[0])
resids.plot(kind='kde', title='Density', ax=ax[1])


plt.show()

#Results show that the mean is nearly zero and that there is uniform variance troughout the data'''

#**Implement AUTO ARIMA Model**

#**Stationarize through Differencing**

In [ ]:
df1 = df["Total Energy Consumption"].diff().dropna()
results = adfuller(df1)
print(results)
#Here, we obtain the same p-value eariler obtained by performing the ADF Test

#**New Train and Test**

In [ ]:
train_new = df1.iloc[:550] #Everything before 2021 
test_new = df1.iloc[550:] #Everything after 2021
#print(train.shape,test.shape)
print(test_new)

#**Stationarize through Differencing**

In [ ]:
warnings.filterwarnings("ignore")
auto_arima_model = auto_arima(train_new,trace=True, suppress_warnings=True)
auto_arima_model.summary()

#**Match to Test Set**

In [ ]:
prediction=pd.DataFrame(auto_arima_model.predict(n_periods=181),index=test_new.index)
prediction.columns=["Predicted Values"]

In [ ]:
plt.figure(figsize=(25,10))
plt.plot(test_new,label="Testing Values")
plt.plot(prediction,label="Predicted Values")
plt.legend(loc='upper left')
plt.show()

#**Evaluation Metrics**

In [ ]:
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual)); mse = mean_squared_error(actual,forecast)
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse,
            'corr':corr, 'minmax':minmax, 'mse':mse})

In [ ]:
x = np.array(prediction)
y = np.array(test_new)
z = np.concatenate(x)
forecast_accuracy(z*1000000,y*1000000)